In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
from PIL import Image as im

In [3]:
df = pd.read_csv("../input/digit-recognizer/train.csv")
df.columns

Index(['label', 'pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5',
       'pixel6', 'pixel7', 'pixel8',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=785)

In [4]:
label = df['label']
df = df.drop('label', axis=1)
df

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
def preprocessing(df):
    images = []
    for index, row in df.iterrows():
        row = row.to_numpy()
        row = row
        row = row.astype(np.uint8)
        row = np.reshape(row,(28,28,1))
        images.append(row)

    images = np.array(images)

    print(images.shape)
    
    return images

images = preprocessing(df)

(42000, 28, 28, 1)


In [6]:
class myCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get('accuracy') > 0.995:
            print("\nAccuracy is high cancelling training.")
            self.model.stop_training = True

In [7]:
def convolutional_model():

    # Define the model, it should have 5 layers:
    # - A Conv2D layer with 32 filters, a kernel_size of 3x3, ReLU activation function
    #    and an input shape that matches that of every image in the training set
    # - A MaxPooling2D layer with a pool_size of 2x2
    # - A Flatten layer with no arguments
    # - A Dense layer with 128 units and ReLU activation function
    # - A Dense layer with 10 units and softmax activation function
    
    model = tf.keras.models.Sequential([ 
        
      tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)),
      tf.keras.layers.MaxPooling2D(2,2),
        
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128,activation='relu'),
      tf.keras.layers.Dense(10,activation='softmax')
        
    ]) 

    # Compile the model
    model.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy']) 
        
    return model

In [8]:
# Save your untrained model
model = convolutional_model()

# Instantiate the callback class
callbacks = myCallback()

# Train your model (this can take up to 5 minutes)
history = model.fit(images, label, epochs=10, callbacks=[callbacks])

2022-02-18 09:49:25.632185: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-02-18 09:49:25.811081: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
1313/1313 [==============================] - 11s 8ms/step - loss: 0.7802 - accuracy: 0.9174
Epoch 2/10
1313/1313 [==============================] - 10s 8ms/step - loss: 0.0948 - accuracy: 0.9715
Epoch 3/10
1313/1313 [==============================] - 11s 8ms/step - loss: 0.0600 - accuracy: 0.9818
Epoch 4/10
1313/1313 [==============================] - 10s 8ms/step - loss: 0.0490 - accuracy: 0.9842
Epoch 5/10
1313/1313 [==============================] - 10s 8ms/step - loss: 0.0371 - accuracy: 0.9886
Epoch 6/10
1313/1313 [==============================] - 11s 8ms/step - loss: 0.0369 - accuracy: 0.9880
Epoch 7/10
1313/1313 [==============================] - 10s 8ms/step - loss: 0.0264 - accuracy: 0.9915
Epoch 8/10
1313/1313 [==============================] - 10s 8ms/step - loss: 0.0257 - accuracy: 0.9921
Epoch 9/10
1313/1313 [==============================] - 11s 9ms/step - loss: 0.0212 - accuracy: 0.9932
Epoch 10/10
1313/1313 [==============================] - 10s 8ms/step - l

In [9]:
test = pd.read_csv("../input/digit-recognizer/test.csv")
test.columns

Index(['pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5', 'pixel6',
       'pixel7', 'pixel8', 'pixel9',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=784)

In [10]:
test_images = preprocessing(test)

y_prob = model.predict(test_images) 
y_classes = y_prob.argmax(axis=-1)
print(type(y_classes))

(28000, 28, 28, 1)
<class 'numpy.ndarray'>


In [11]:
sub = pd.DataFrame(y_classes, columns = ['Label'])
sub['ImageId'] = np.arange(1,len(sub)+1)
sub = sub[['ImageId','Label']]
sub.to_csv('submission.csv', index=False)